<a href="https://colab.research.google.com/github/MartinSantaGitHub/curso-series-temporales/blob/master/11.%20Auto%20Arima/Python/15_Auto_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [5]:
!pip install arch

In [6]:
!pip install pmdarima

In [7]:
!pip install yfinance

In [8]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [9]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2020-03-20", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [10]:
df_comp = raw_data.copy()

In [11]:
df_comp.head()

^N225                ...       ^GDAXI       
                    Open          High  ...        Close Volume
Date                                    ...                    
1994-01-07  17842.980469  18131.410156  ...  2224.949951    0.0
1994-01-10  18186.519531  18567.060547  ...  2225.000000    0.0
1994-01-11  18481.849609  18671.669922  ...  2228.100098    0.0
1994-01-12  18447.339844  18807.080078  ...  2182.060059    0.0
1994-01-13  18770.380859  18823.380859  ...  2142.370117    0.0

[5 rows x 20 columns]

In [12]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [13]:
df_comp.head()

^N225                ...         ftse        nikkei
                    Open          High  ...                           
Date                                    ...                           
1994-01-07  17842.980469  18131.410156  ...  3446.000000  18124.009766
1994-01-10  18186.519531  18567.060547  ...  3440.600098  18443.439453
1994-01-11  18481.849609  18671.669922  ...  3413.800049  18485.250000
1994-01-12  18447.339844  18807.080078  ...  3372.000000  18793.880859
1994-01-13  18770.380859  18823.380859  ...  3360.000000  18577.259766

[5 rows x 24 columns]

In [14]:
df_comp.tail()

^N225                ...         ftse        nikkei
                    Open          High  ...                           
Date                                    ...                           
2020-03-13  18183.470703  18184.460938  ...  5366.100098  17431.050781
2020-03-16  17586.080078  17785.759766  ...  5151.100098  17002.039062
2020-03-17  16726.949219  17557.039062  ...  5294.899902  17011.529297
2020-03-18  17154.080078  17396.839844  ...  5080.600098  16726.550781
2020-03-19  16995.769531  17160.970703  ...  5151.600098  16552.830078

[5 rows x 24 columns]

In [15]:
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [16]:
df_comp.head()

,spx,dax,ftse,nikkei
,,,,
Date,,,,
1994-01-07,469.899994,2224.949951,3446.000000,18124.009766
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766


### Creating Returns

In [17]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [18]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [19]:
from pmdarima.arima import auto_arima

In [20]:
model_auto = auto_arima(df.ret_ftse[1:])

In [21]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(3, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [22]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5467
Model:               SARIMAX(3, 0, 5)   Log Likelihood               -8452.802
Date:                Sat, 24 Apr 2021   AIC                          16925.604
Time:                        15:29:22   BIC                          16991.669
Sample:                             0   HQIC                         16948.654
                               - 5467                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0396      0.033      1.202      0.229      -0.025       0.104
ar.L1         -0.1462      0.077     -1.906      0.057      -0.297       0.004
ar.L2         -0.8669      0.020    -43.185      0.000      -0.906      -0.828
ar.L3         -0.2992      0.070     -4.273      0.000      -0.437      -0.162
ma.L1          0.1227      0.077      1.596      0.110      -0.028       0.273
ma.L2          0.8146      0.023     35.803      0.000       0.770       0.859
ma.L3          0.1931      0.069      2.805      0.005       0.058       0.328
ma.L4         -0.0093      0.009     -1.071      0.284      -0.026       0.008
ma.L5         -0.1066      0.008    -13.240      0.000      -0.122      -0.091
sigma2         1.2879      0.013     96.048      0.000       1.262       1.314
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              7358.27
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               1.81   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.67
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [23]:
model_auto2 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample_size = int(len(df_comp)*0.2))


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually - "ctt" cuadratic - boolean matrix, more complex
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [24]:
model_auto2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 6834
Model:             SARIMAX(0, 0, 1)x(4, 0, [], 5)   Log Likelihood               -6809.273
Date:                            Sat, 24 Apr 2021   AIC                          13640.547
Time:                                    15:49:25   BIC                          13715.673
Sample:                                         0   HQIC                         13666.465
                                           - 6834                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0008      0.015     -0.052      0.959      -0.031       0.029
drift      -2.401e-06   4.33e-06     -0.555      0.579   -1.09e-05    6.08e-06
x1             0.0872      0.006     15.219      0.000       0.076       0.098
x2             0.5624      0.005    110.721      0.000       0.552       0.572
x3             0.0720      0.004     16.452      0.000       0.063       0.081
ma.L1         -0.1165      0.008    -15.498      0.000      -0.131      -0.102
ar.S.L5       -0.0332      0.009     -3.776      0.000      -0.050      -0.016
ar.S.L10      -0.0503      0.010     -5.225      0.000      -0.069      -0.031
ar.S.L15      -0.0349      0.009     -3.844      0.000      -0.053      -0.017
ar.S.L20      -0.0233      0.009     -2.521      0.012      -0.041      -0.005
sigma2         0.4641      0.005     99.300      0.000       0.455       0.473
===================================================================================
Ljung-Box (L1) (Q):                   3.16   Jarque-Bera (JB):             15206.90
Prob(Q):                              0.08   Prob(JB):                         0.00
Heteroskedasticity (H):               0.48   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.29
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""